In [1]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

In [4]:
import pandas as pd
from cobra.io import read_sbml_model

model = read_sbml_model("../model/Rpom_05.xml")

https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http

In [43]:
import pickle
from model_building.biocyc_update_pipeline.transform.build_templates import get_bounds

# Get bounds for all reactions according to either database

with open("../model_building/biocyc_update_pipeline/data/raw_reaction_data__RUEGERIA_POMEROYI_DSS3.pkl", "rb") as f:
    db1 = pickle.load(f)

with open("../model_building/biocyc_update_pipeline/data/raw_reaction_data__GCF_000011965.pkl", "rb") as f:
    db2 = pickle.load(f)

def to_float_tuple(int_tuple):
    if int_tuple is None:
        return None
    return tuple([float(x) for x in int_tuple])

db1_bounds = pd.DataFrame([
    {"ID" : rxn_data["@frameid"], "Bounds": to_float_tuple(get_bounds(rxn_data))}
    for rxn_data in db1
])

db2_bounds = pd.DataFrame([
    {"ID" : rxn_data["@frameid"], "Bounds": to_float_tuple(get_bounds(rxn_data))}
    for rxn_data in db2
])

/home/myang42/projects/rpom-fba/notebooks/../model_building/biocyc_update_pipeline/transform/build_templates.py:197: UserWarning: No reaction direction specified (inference from enzymatic reactions not yet implemented)
  warnings.warn(


In [44]:
# Subset to reactions in the model
db1_bounds_in_model = db1_bounds[[rxnid in model.reactions for rxnid in db1_bounds["ID"]]]
db2_bounds_in_model = db2_bounds[[rxnid in model.reactions for rxnid in db2_bounds["ID"]]]

In [45]:
# Merge with model bounds, flux on glucose

with model:
    model.reactions.get_by_id("EX_glc").bounds = (-10., 0.)
    sol = model.optimize()

model_bounds = pd.DataFrame([
    {"ID": rxn.id, "Model Bounds": rxn.bounds, "Flux" : sol[rxn.id]}
    for rxn in model.reactions])

all_bounds = db1_bounds_in_model.merge(
    db2_bounds_in_model,
    left_on="ID", right_on="ID",
    suffixes=[" db1", " db2"]).merge(
        model_bounds,
        left_on="ID", right_on="ID")

all_bounds

,ID,Bounds db1,Bounds db2,Model Bounds,Flux
0,GSAAMINOTRANS-RXN,"(0.0, 1000.0)","(0.0, 1000.0)","(0.0, 1000.0)",0.000000
1,MALEYLACETOACETATE-ISOMERASE-RXN,"(0.0, 1000.0)","(0.0, 1000.0)","(0.0, 1000.0)",0.000000
2,RXN0-5462,"(0.0, 1000.0)","(0.0, 1000.0)","(0.0, 1000.0)",0.000000
3,RXN-11741,"(0.0, 1000.0)","(0.0, 1000.0)","(0.0, 1000.0)",0.000000
4,RXN-9530,None,"(-1000.0, 0.0)","(-1000.0, 0.0)",0.000000
...,...,...,...,...,...
818,RXN0-2141,None,"(0.0, 1000.0)","(-1000.0, 1000.0)",0.000000
819,2.1.1.132-RXN,"(0.0, 1000.0)","(0.0, 1000.0)","(0.0, 1000.0)",0.000000
820,RXN-9644,"(0.0, 1000.0)","(0.0, 1000.0)","(0.0, 1000.0)",0.000000
821,DEPHOSPHOCOAKIN-RXN,"(0.0, 1000.0)","(0.0, 1000.0)","(0.0, 1000.0)",0.000565


In [46]:
# How many reactions are in one db but not the other?
all_bounds[all_bounds["Bounds db1"].isna() ^ all_bounds["Bounds db2"].isna()].sort_values("Flux")

,ID,Bounds db1,Bounds db2,Model Bounds,Flux
384,RXN-5901,None,"(-1000.0, 0.0)","(-1000.0, 0.0)",-2.027874
248,RIBULP3EPIM-RXN,None,"(-1000.0, 1000.0)","(-1000.0, 1000.0)",-0.383860
270,2TRANSKETO-RXN,None,"(-1000.0, 1000.0)","(-1000.0, 1000.0)",-0.347433
664,RXN-9515,None,"(-1000.0, 0.0)","(-1000.0, 0.0)",-0.272897
221,RXN-9526,None,"(-1000.0, 0.0)","(-1000.0, 0.0)",-0.272897
651,RXN-9534,None,"(-1000.0, 0.0)","(-1000.0, 0.0)",-0.272897
5,RXN-9542,None,"(-1000.0, 0.0)","(-1000.0, 0.0)",-0.272897
206,RXN-9538,None,"(-1000.0, 0.0)","(-1000.0, 0.0)",-0.272897
454,1TRANSKETO-RXN,None,"(-1000.0, 1000.0)","(-1000.0, 1000.0)",-0.036428
665,RXN-12726,None,"(0.0, 1000.0)","(-1000.0, 1000.0)",0.000000


In [62]:
# Highlight bounds disagreements
from itertools import pairwise

# Convoluted syntax - basically, it's a problem when the bounds aren't all equal,
# BUT we can ignore any missing bounds (e.g. if db1 says (0.0, 1000.0) while db2
# has no bounds for that reaction, that's not necessarily an issue worth highlighting.)
bonunds_issues = all_bounds[[
    not all([x == y or x is None or y is None
             for x, y in pairwise([bounds_db1, bounds_db2, bounds_model])])
    for _, (_, bounds_db1, bounds_db2, bounds_model, _) in all_bounds.iterrows()
]].sort_values("Flux")
bonunds_issues

,ID,Bounds db1,Bounds db2,Model Bounds,Flux
596,FORMATETHFLIG-RXN,"(-1000.0, 1000.0)","(0.0, 1000.0)","(-1000.0, 1000.0)",-8.624372
207,METHYLMALONYL-COA-EPIM-RXN,"(-1000.0, 0.0)","(-1000.0, 0.0)","(-1000.0, 1000.0)",-1.114003
187,BRANCHED-CHAINAMINOTRANSFERILEU-RXN,"(-1000.0, 1000.0)","(-1000.0, 0.0)","(-1000.0, 1000.0)",-0.236940
444,RXN-12567,None,"(-1000.0, 0.0)","(-1000.0, 1000.0)",0.000000
629,5.5.1.2-RXN,"(-1000.0, 0.0)","(-1000.0, 0.0)","(0.0, 1000.0)",0.000000
...,...,...,...,...,...
25,MALYL-COA-LYASE-RXN,None,"(-1000.0, 0.0)","(0.0, 1000.0)",1.114659
707,GCVMULTI-RXN,None,"(-1000.0, 1000.0)","(0.0, 1000.0)",4.258326
338,PSERTRANSAM-RXN,"(-1000.0, 0.0)","(-1000.0, 1000.0)","(0.0, 1000.0)",5.357923
519,PYRNUTRANSHYDROGEN-RXN,"(0.0, 1000.0)","(-1000.0, 1000.0)","(0.0, 1000.0)",8.025592


In [64]:
bonunds_issues.to_clipboard(index=False)

In [ ]:
model.reactions.get_by_id("DIHYDROFOLATEREDUCT-RXN")
sol["DIHYDROFOLATEREDUCT-RXN"]

0.0006565883615579256

In [111]:
model.reactions.get_by_id("DIHYDROFOLATESYNTH-RXN")
sol["DIHYDROFOLATESYNTH-RXN"]

0.0006565883615579256

In [107]:
model.reactions.get_by_id("DIHYDROFOLATESYNTH-RXN")

Reaction identifier,DIHYDROFOLATESYNTH-RXN
Name,DIHYDROFOLATESYNTH-RXN
Memory address,0x7ae5ad409880
Stoichiometry,"7-8-DIHYDROPTEROATE[c] + ATP[c] + GLT[c] --> ADP[c] + DIHYDROFOLATE[c] + PROTON[c] + Pi[c] 7,8-dihydropteroate + ATP + L-glutamate --> ADP + 7,8-dihydrofolate monoglutamate + H+ + phosphate"
GPR,
Lower bound,0.0
Upper bound,1000.0
